In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [2]:
group_dict = pickle.load(open('groups_dict','rb'))
df_historical = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

*CALCULAR TEAM OVERALL*

In [7]:
df_historical

,HomeTeam,AwayTeam,Year,HomeGoals,AwayGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Uruguay,Argentina,1930,4,2,6
2,Uruguay,Yugoslavia,1930,6,1,7
3,Argentina,United States,1930,6,1,7
4,Paraguay,Belgium,1930,1,0,1
...,...,...,...,...,...,...
895,Brazil,Costa Rica,2018,2,0,2
896,Serbia,Switzerland,2018,1,2,3
897,Serbia,Brazil,2018,0,2,2
898,France,Peru,2018,1,0,1


In [8]:
df_home = df_historical[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [9]:
df_home = df_home.rename(columns={'HomeTeam': 'Team', 'HomeGoals': 'Scored', 'AwayGoals': 'Conceded'})
df_away = df_away.rename(columns={'AwayTeam': 'Team', 'HomeGoals': 'Conceded', 'AwayGoals': 'Scored'})

In [13]:
df_team_overall = pd.concat([df_home, df_away], ignore_index=True).groupby('Team').mean()
df_team_overall

,Scored,Conceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.691358,1.148148
Australia,0.812500,1.937500
Austria,1.482759,1.620690
...,...,...
Uruguay,1.553571,1.321429
Wales,0.800000,0.800000
West Germany,2.112903,1.241935


In [14]:
def predict_points(home, away):
    if home in df_team_overall.index and away in df_team_overall.index:
        home_l = df_team_overall.at[home, 'Scored'] * df_team_overall.at[away, 'Conceded']
        away_l = df_team_overall.at[away, 'Scored'] * df_team_overall.at[home, 'Conceded']
        # ph -> prob home, pa -> prob away, pdr -> prob draw
        ph, pa, pdr = 0, 0, 0
        for h in range (0, 11):
            for a in range(0, 11):
                p = poisson.pmf(h, home_l) * poisson.pmf(a, away_l)

                if h == a:
                    pdr += p
                elif h > a:
                    ph += p
                else:
                    pa += p
        
        points_home = 3 * ph + pdr
        points_away = 3 * pa + pdr

        return (points_home, points_away)
    else:
        return (0, 0)

In [15]:
print(predict_points('Spain', 'Costa Rica'))
print(predict_points('Argentina', 'Poland'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.118647233657988, 0.6998438033414867)
(1.7989857064343988, 0.9988537381758329)
(0, 0)
